In [2]:
import numpy as np
import itertools
from typing import List, Tuple

In [9]:
def set_diag_vector(diagonal_length: int) -> List[int]:
    if diagonal_length <= 1:
        raise ValueError("diagonal_length must be positive and greater than one.")
    if diagonal_length == 1:
         return [1]
    return [1] + [0] * (diagonal_length-1)


In [15]:
def set_on_off_measurement_one_outcome(outcome: int, cutoff_dim: int, diag_vector: List[int] = None) -> np.ndarray:
    if outcome != 0 and outcome != 1:
        raise ValueError("outcome must be either 0 or 1")
    if diag_vector is None:
        diag_vector = set_diag_vector(diagonal_length=cutoff_dim+1)
    if outcome == 0:
        return np.diag(diag_vector)
    return np.identity(n=cutoff_dim+1) - np.diag(diag_vector)
    

In [14]:
set_on_off_measurement_one_outcome(outcome=1, cutoff_dim=2)

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [16]:
def generate_all_outcomes(modes=int) -> List[Tuple[int]]:
    options = [0, 1]
    return list(itertools.product(options, repeat=modes))


In [17]:
generate_all_outcomes(modes=2)

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [18]:
def compute_kronecker_matrices(matrices: List[np.ndarray]) -> np.ndarray:
    if len(matrices) < 2:
        raise ValueError("The number of matrices must be at least 2")
    if len(matrices) == 2:
        return np.kron(matrices[0], matrices[1])
    last_matrix = matrices.pop()
    return np.kron(compute_kronecker_matrices(matrices), last_matrix)


In [23]:
def generate_on_off_measurement_one_outcome(one_outcome: Tuple[int], cutoff_dim: int, diag_vector: List[int] = None) -> np.ndarray:
    if diag_vector is None:
        diag_vector = set_diag_vector(diagonal_length=cutoff_dim + 1)
    one_mode_measurements = [set_on_off_measurement_one_outcome(
        outcome=outcome_i, cutoff_dim=cutoff_dim, diag_vector=diag_vector) for outcome_i in one_outcome]
    return compute_kronecker_matrices(matrices=one_mode_measurements)

In [24]:
def generate_all_on_off_measurements(modes: int, cutoff_dim: int) -> List[np.ndarray]:
    outcomes = generate_all_outcomes(modes=modes)
    diag_vector = set_diag_vector(diagonal_length=cutoff_dim + 1)
    
    return [generate_on_off_measurement_one_outcome(one_outcome=outcome, cutoff_dim=cutoff_dim, diag_vector=diag_vector) 
            for outcome in outcomes]

In [25]:
generate_all_on_off_measurements(modes=2, cutoff_dim=1)

[array([[1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]),
 array([[0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.]])]

In [29]:
generate_all_on_off_measurements(modes=2, cutoff_dim=2)


[array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0.

In [39]:
import strawberryfields as sf
from strawberryfields.backends import BaseFockState

In [34]:
alphas = [0.7]
betas = [0.3]
modes = 2
cutoff_dim = 2

In [33]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": cutoff_dim})
prog = sf.Program(modes)

par_alpha = prog.params("alpha")
par_beta = prog.params("beta")

with prog.context as q:
    sf.ops.Dgate(par_alpha) | q[0]
    sf.ops.Dgate(par_beta) | q[1]
    sf.ops.BSgate() | (q[0], q[1])


In [35]:
result = eng.run(prog, args={
    "alpha": alphas[0],
    "beta": betas[0],
})

In [36]:
measurements = generate_all_on_off_measurements(modes=modes, cutoff_dim=cutoff_dim)

In [38]:
measurements[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [41]:
state =result.state

In [54]:
state.dm()

array([[[[5.59898367e-01+0.j, 3.95907932e-01+0.j],
         [3.95907932e-01+0.j, 2.79949183e-01+0.j]],

        [[1.58363173e-01+0.j, 2.61077064e-17+0.j],
         [1.11979673e-01+0.j, 1.84609363e-17+0.j]]],


       [[[1.58363173e-01+0.j, 1.11979673e-01+0.j],
         [2.61077064e-17+0.j, 1.84609363e-17+0.j]],

        [[4.47918693e-02+0.j, 7.38437450e-18+0.j],
         [7.38437450e-18+0.j, 1.21738583e-33+0.j]]]])

In [53]:
state.dm().shape

(2, 2, 2, 2)

In [49]:
sum([result.state.fock_prob([0,0]),
     result.state.fock_prob([0,1]),
     result.state.fock_prob([1,0]),
     result.state.fock_prob([1,1])])

0.8846394191733352

In [61]:
ket = state.data

In [63]:
bra = np.conjugate(ket)

In [69]:
density_matrix = np.outer(ket, bra)

In [72]:
np.inner(density_matrix, measurements[0])


ValueError: shapes (4,4) and (9,9) not aligned: 4 (dim 1) != 9 (dim 0)

In [71]:
density_matrix.shape

(4, 4)

In [574]:
def compute_probability_all_outcomes(state: BaseFockState, ops_measurements: List[LinearOperator]) -> List[EagerTensor]:
    ket_op = tf.linalg.LinearOperatorFullMatrix(state.data)
    bra_op = tf.linalg.LinearOperatorAdjoint(ket_op)
    dm_op = tf.linalg.LinearOperatorKronecker([ket_op, bra_op])
    print(dm_op.to_dense())
    return [tf.cast(op_measurement.matmul(dm_op).trace(), dtype=tf.float32) for op_measurement in ops_measurements]


In [575]:
compute_probability_all_outcomes(state=result.state, ops_measurements=ops_measurements)


tf.Tensor(
[[0.5598983 +0.j 0.15836315+0.j 0.39590788+0.j 0.11197966+0.j]
 [0.39590788+0.j 0.        +0.j 0.27994916+0.j 0.        +0.j]
 [0.15836315+0.j 0.04479186+0.j 0.        +0.j 0.        +0.j]
 [0.11197966+0.j 0.        +0.j 0.        +0.j 0.        +0.j]], shape=(4, 4), dtype=complex64)


[<tf.Tensor: shape=(), dtype=float32, numpy=0.5598983>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>]

In [435]:
tf.reduce_sum(compute_probability_all_outcomes(state=result.state, ops_measurements=ops_measurements))


<tf.Tensor: shape=(), dtype=float32, numpy=0.5598983>

In [571]:
result.state.fock_prob([0,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.55989826>

In [434]:
tf.reduce_sum([result.state.fock_prob([0,0]),
              result.state.fock_prob([0,1]),
              result.state.fock_prob([1,0]),
              result.state.fock_prob([1,1])])

<tf.Tensor: shape=(), dtype=float32, numpy=0.88463926>

In [577]:
result.state.dm()

<tf.Tensor: shape=(2, 2, 2, 2), dtype=complex64, numpy=
array([[[[0.5598983 +0.j, 0.39590788+0.j],
         [0.39590788+0.j, 0.27994916+0.j]],

        [[0.15836315+0.j, 0.        +0.j],
         [0.11197966+0.j, 0.        +0.j]]],


       [[[0.15836315+0.j, 0.11197966+0.j],
         [0.        +0.j, 0.        +0.j]],

        [[0.04479186+0.j, 0.        +0.j],
         [0.        +0.j, 0.        +0.j]]]], dtype=complex64)>